In [26]:
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
from matplotlib.widgets import Slider
import matplotlib.patheffects as path_effects
import os
from astropy.io import fits
from glob import glob
import pandas as pd
import numpy as np

In [29]:
class DataCube:
	"""
	An object combining multiple 1D and 2D spectra from a given folder.
	
	
	Parameters :
	----------
	folder :
		path to the folder containing the final data. Will loop on every fits file 
		and keep the one containing the _s2d or _x1d suffix
	
	Properties : 
	----------
	table : 
		a table used for making a correspondence between a source id, a s2d file and a x1d file. 
		Those files are stored as a list of paths 
	"""
	
	def __init__(self, folder):

		s2dList = [[file] for file in sorted(glob(os.path.join(folder, '*_s2d*'))) if (len(fits.open(file)[1].header.get("SHUTSTA", "")) > 1)]
		x1dList = [[x if os.path.exists(x := file[0].replace("_s2d", "_x1d")) else [None]] for file in s2dList]
		#s2dList = [[file] for file in sorted(glob(os.path.join(folder, '*_s2d*')))]
		#x1dList = [[x] if os.path.exists(x := file[0].replace("_s2d", "_x1d")) else [None] for file in s2dList]
		sourceList = [fits.open(file[0])[1].header["SOURCEID"] for file in s2dList]
		
		self.table = pd.DataFrame({"sourceID": sourceList, "s2d": s2dList, "x1d": x1dList})

		# Initializes the dataframe which will contain the data models
		self.dataTable = pd.DataFrame({"sourceID" : [], "s2d" : [], "x1d" : []})

		
		
	def combineDataCube(self, datacube,i=1):
		"""
		Combines 2 Datacubes
		Parameters
		----------
		datacube :
			Another datacube to be appended to this one, or None, which will fill append a none to every list in the array
		i : how many Nones to add. This depends on if it's the 1st call of the function or the 2nd
		"""
		
		print("Starting Combining Datacubes")

		if datacube is None:
			for i in range(len(self.table)):
				self.table["s2d"][i].append(None)
				self.table["x1d"][i].append(None)
		else:
			print("Merging...")
			# Perform an outer join to include all sourceIDs from both DataFrames
			merged = self.table.merge(
				datacube.table, on="sourceID", how="outer", suffixes=("_self", "_other")
			)

			print("Replacing empty values...")
			# Fill missing values in 's2d' and 'x1d' columns with empty lists
			merged["s2d_self"] = merged["s2d_self"].apply(lambda x: x if isinstance(x, list) else [None for _ in range(i)])
			merged["s2d_other"] = merged["s2d_other"].apply(lambda x: x if isinstance(x, list) else [None])
			merged["x1d_self"] = merged["x1d_self"].apply(lambda x: x if isinstance(x, list) else [None for _ in range(i)])
			merged["x1d_other"] = merged["x1d_other"].apply(lambda x: x if isinstance(x, list) else [None])

			print("Appending paths...")
			# Combine the 's2d' and 'x1d' columns
			merged["s2d"] = merged["s2d_self"] + merged["s2d_other"]
			merged["x1d"] = merged["x1d_self"] + merged["x1d_other"]

			# Keep only necessary columns: 'sourceID', 's2d', 'x1d'
			self.table = merged[["sourceID", "s2d", "x1d"]]
		
		print("Finished Combining Datacubes!")

	def preloadDataCube(self):
		"""
		Initializes self.dataTable, a table structurally identical to self.table, 
		except the paths are replaced by the corresponding datamodels
		"""
		print("Starting loading data...")
		print("Copying...")
		self.dataTable = self.table.copy()
		
		# Process lists of file paths
		def processList(fileList):
			return [fits.open(file) if isinstance(file, str) else None for file in fileList]
		
		print("Loading...")
		# Process the 'x1d' and 's2d' columns
		self.dataTable["x1d"] = self.dataTable["x1d"].apply(processList)
		self.dataTable["s2d"] = self.dataTable["s2d"].apply(processList)

		for idx,s2ds in enumerate(self.dataTable["s2d"]):
			for j in range(len(s2ds)):
				toJy = 1
				s2d = s2ds[j]
				if s2d is None:
					continue
				if s2d[1].header["BUNIT"] == "MJy/sr" :
					toJy = s2d[1].header["PIXAR_SR"]*1e6
					s2d[1].header["BUNIT"] = "Jy"
					print("MJy/sr")
				if s2d[1].header["BUNIT"] == "MJy" :
					toJy = 1e6
					s2d[1].header["BUNIT"] = "Jy"
					print("MJy")
				x1d = self.dataTable["x1d"][idx][j]
				x1d[1].data["FLUX"] *= toJy
				x1d[1].data["FLUX_ERROR"] *= toJy
		
		print("Finished loading data!")

	@staticmethod
	def exploreDataCube(dc):
		fig, axes = plt.subplots(4, 1, figsize=(18, 7), gridspec_kw={'height_ratios': [1, 1, 1, 8]})
		plt.subplots_adjust(left=0.1, bottom=0.15, right=0.9, top=0.9, hspace=0)

		idx = 0

		def drawExtraction(axe, x1d):
			x0 = x1d.header["EXTRXSTR"]
			x1 = x1d.header["EXTRXSTP"]
			y0 = x1d.header["EXTRYSTR"]
			y1 = x1d.header["EXTRYSTP"]
			axe.vlines((x0, x1), (y0, y0), (y1, y1), color='r', linestyles='dashed', linewidth=0.5)
			axe.hlines((y0, y1), (x0, x0), (x1, x1), color='r', linestyles='dashed', linewidth=0.5)

		# Update sourceID
		def update(val):
			idx = int(slider.val)  # Get the current slider value

			axes[-1].clear()  # Clear the current error bar plot

			legends = ["n=0.25", "n=1", "Default Pipeline"]
			colors = ["blue", "orange", "green"]
			for i in range(len(legends)):
				axes[i].clear()
				if not dc.dataTable["s2d"][idx][i] is None:
					z1, z2 = ZScaleInterval().get_limits(dc.dataTable["s2d"][idx][i][1].data)
					axes[i].imshow(dc.dataTable["s2d"][idx][i][1].data, aspect='auto', vmin=z1, vmax=z2, cmap="viridis", origin="lower", interpolation="none")
					drawExtraction(axes[i], dc.dataTable["x1d"][idx][i][1])

				text = axes[i].text(0.02, 0.3, legends[i], color="w", transform=axes[i].transAxes)
				text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='black'), path_effects.Normal()])

				if not dc.dataTable["x1d"][idx][i] is None:
					wavelength = dc.dataTable["x1d"][idx][i][1].data["WAVELENGTH"]
					flux = dc.dataTable["x1d"][idx][i][1].data["FLUX"]
					err = dc.dataTable["x1d"][idx][i][1].data["FLUX_ERROR"]
					ratio = np.nanmax(dc.dataTable["x1d"][idx][2][1].data["FLUX_ERROR"]) / np.nanmax(err)
					flux *= ratio
					err *= ratio
					axes[-1].plot(wavelength, flux, label=legends[i], color=colors[i])
					axes[-1].plot(wavelength, err, color=colors[i], alpha=0.5, linewidth=0.5)

			axes[-1].text(0.05, 0.05, f"SourceID: {dc.dataTable['sourceID'][idx]}", color="k", transform=axes[-1].transAxes, size=15)
			axes[-1].set_xlabel(r"$\lambda$ (µm)")
			axes[-1].set_ylabel(r"Flux (Jy)")
			axes[-1].legend()
			axes[-1].grid()
			fig.canvas.draw_idle()


		def onKey(event):
			current = slider.val
			if event.key == "right":  # Move slider one step right
				new = min(current + 1, N - 1)  # Ensure within bounds
				slider.set_val(new)
			elif event.key == "left":  # Move slider one step left
				new = max(current - 1, 0)  # Ensure within bounds
				slider.set_val(new)

		# Slider
		ax_slider = plt.axes((0.2, 0.05, 0.6, 0.03))
		N = len(dc.dataTable["sourceID"])
		slider = Slider(ax_slider, 'Source', 0, N - 1, valinit=idx, valstep=1)
		update(idx)

		# Attach the update function to the slider
		slider.on_changed(update)

		# Connect keypress handler
		fig.canvas.mpl_connect("key_press_event", onKey)

		# Show the plot
		plt.show()


In [30]:
dc = DataCube("/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final")
dc.combineDataCube(DataCube("/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/BNBG-n0.25/Final"))
dc.combineDataCube(DataCube("/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Default/Final"))

dc.table

Starting Combining Datacubes
Merging...
Replacing empty values...
Appending paths...
Finished Combining Datacubes!
Starting Combining Datacubes
Merging...
Replacing empty values...
Appending paths...
Finished Combining Datacubes!


,sourceID,s2d,x1d
0,-168,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
1,-167,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
2,-166,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
3,-165,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
4,-163,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
...,...,...,...
311,43262,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
312,43450,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
313,43461,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
314,44055,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...


In [31]:
dc.preloadDataCube()
dc.dataTable

Starting loading data...
Copying...
Loading...
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
MJy/sr
M

,sourceID,s2d,x1d
0,-168,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
1,-167,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
2,-166,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
3,-165,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
4,-163,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
...,...,...,...
311,43262,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
312,43450,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
313,43461,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...
314,44055,[[<astropy.io.fits.hdu.image.PrimaryHDU object...,[[<astropy.io.fits.hdu.image.PrimaryHDU object...


In [32]:
%matplotlib Qt5Agg

In [34]:
DataCube.exploreDataCube(dc)